I just realised that the time series data is only dependent on water velocity as we cannot easily add consecutive time series microplastics readings in addition to the image. So we'll do the API calls and everything to get time series data points as standalone training samples. Training the model will entail inputting time series data of multiple images of days before the reading, but not including the image on the day of the reading itself. This will prompt the model to "predict" the future, in a way, and make the microplastic prediction without needing the water velocity of the day itself.

In [35]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, shape
from datetime import timedelta, datetime

import ee
import json

import numpy as np
import pickle

In [2]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

Read in microplastics data.

In [14]:
df = gpd.read_file(r"data_files/DataRecords.csv")

df['Longitude'] = df['Longitude'].astype('float')
df['Latitude'] = df['Latitude'].astype('float')

df['geometry'] = df.apply(lambda x: Point(x['Longitude'], x['Latitude']), axis=1)
df = df[['Date', 'geometry', 'Microplastics Measurement (density)']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.rename(columns={'Microplastics Measurement (density)':'Density'})
df['Density'] = df['Density'].astype('float')
df.crs = "EPSG:4326"
df

C:\Users\weipy\anaconda3\envs\splash-awards\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Date,geometry,Density
0,1992-11-01,POINT (-59.35000 18.83000),0.004320
1,1992-11-01,POINT (-59.35000 18.83000),0.034556
2,1992-11-02,POINT (-59.43000 17.80000),0.008640
3,1992-11-02,POINT (-59.43000 17.80000),0.000000
4,1992-11-02,POINT (-59.57000 17.03000),0.000000
...,...,...,...
10715,2015-06-04,POINT (3.24000 42.02000),0.036000
10716,2015-06-05,POINT (3.29000 42.37000),1.004000
10717,2015-06-05,POINT (3.06000 41.75000),0.335000
10718,2015-06-05,POINT (2.78000 41.62000),1.986000


HYCOM only has data from after 1992-10-02, get rid of entries before that.

In [15]:
df = df[df['Date'] > '1992-10-02']
df

,Date,geometry,Density
0,1992-11-01,POINT (-59.35000 18.83000),0.004320
1,1992-11-01,POINT (-59.35000 18.83000),0.034556
2,1992-11-02,POINT (-59.43000 17.80000),0.008640
3,1992-11-02,POINT (-59.43000 17.80000),0.000000
4,1992-11-02,POINT (-59.57000 17.03000),0.000000
...,...,...,...
10715,2015-06-04,POINT (3.24000 42.02000),0.036000
10716,2015-06-05,POINT (3.29000 42.37000),1.004000
10717,2015-06-05,POINT (3.06000 41.75000),0.335000
10718,2015-06-05,POINT (2.78000 41.62000),1.986000


Calculate the region of interest of the subset of readings. Randomly sample a subset of the data to api request. 

In [16]:
df = df.sample(n=2001, random_state=0)
df = df.to_crs(epsg=3857)

# cap_style=3 indicates square buffer
df['Region'] = df.buffer(10e3, cap_style=3)

df = df.set_geometry('Region')
df = df.drop(columns=['geometry'])

df = df.to_crs(epsg=4326)

df = df.reset_index(drop=True)

df

,Date,Density,Region
0,2002-12-13,0.000000,"POLYGON ((-116.03017 16.91596, -116.03017 16.7..."
1,2002-02-20,0.122376,"POLYGON ((-71.89017 24.97146, -71.89017 24.808..."
2,1997-11-13,0.000000,"POLYGON ((-61.03017 14.06715, -61.03017 13.892..."
3,2015-01-30,0.005000,"POLYGON ((-99.07017 14.03717, -99.07017 13.862..."
4,2008-12-11,0.008640,"POLYGON ((-151.54017 33.54490, -151.54017 33.3..."
...,...,...,...
1996,2005-06-09,0.034556,"POLYGON ((-69.54017 43.70498, -69.54017 43.574..."
1997,2016-11-08,0.001000,"POLYGON ((116.17983 -8.26111, 116.17983 -8.438..."
1998,2017-11-22,24.000000,"POLYGON ((1.66983 -37.15844, 1.66983 -37.30149..."
1999,2009-08-06,0.165000,"POLYGON ((-131.33017 32.25600, -131.33017 32.1..."


Actual API call. The lookback_days is how many days back the time series will look, exclusive of the date of the reading itself.

In [6]:
def extract_raster_values_from_df(df, image_collection, band_names, task_name, lookback_days=3, scale=5000, export=False):
    df['ee_region'] = df.geometry.apply(lambda x: ee.Geometry.Polygon(list((x.exterior.coords)), proj='EPSG:4326'))
    regionCollection = ee.List([])
    emptyCol = ee.FeatureCollection(ee.Feature(None))
    
    # iterate through all entries
    for row in df.itertuples():
        # instantiate the image collection for the selected time series
        date = getattr(row, 'Date')
        collection = ee.ImageCollection(image_collection).filterDate((date - timedelta(days=lookback_days)).strftime('%Y-%m-%d'), date.strftime('%Y-%m-%d')).select(band_names)
        
        # get the raster information based on the buffer region we created
        pixelInfoRegion = collection.getRegion(geometry=getattr(row, 'ee_region'), scale=scale)

        # remove the first element, which is the header ['id', 'longitude', 'latitude', 'time', 'velocity_u_0', 'velocity_v_0']
        pixelInfoRegion = pixelInfoRegion.remove(pixelInfoRegion.get(0))
        # convert the 2d list of information to features. Going to hard code this part as I cba
        def func(x):
            x = ee.List(x)
            feat = ee.Feature(ee.Geometry.Point([x.get(1), x.get(2)], proj='EPSG:4326'), {'time':x.get(3),'velocity_u_0':x.get(4),'velocity_v_0':x.get(5)})
            return feat
        col = ee.FeatureCollection(pixelInfoRegion.map(func))
        # add the region that we created to the overall feature collection
        
        # ensure collection is not empty bands
        regionCollection = regionCollection.add(ee.Algorithms.If(collection.size(), col, emptyCol))
    
    if export:
        return ee.batch.Export.table.toDrive(collection=ee.FeatureCollection(regionCollection).flatten(), description=task_name, fileFormat="csv", folder="exported_files")
    else:
        return ee.FeatureCollection(regionCollection).flatten()

Doing the full api call crashes the kernel. Split it up into smaller subtasks.

In [7]:
size = 500
for i in range(0, 2000, size):
    task_name = f"water_velocities_{i}"
    task = extract_raster_values_from_df(df.iloc[i:i+size].copy(), 'HYCOM/sea_water_velocity', ['velocity_u_0', 'velocity_v_0'], task_name, export=True)
    task.start()

After that, load the data into dataframes.

In [104]:
df_list = []
for i in range(0, 2000, 500):
    sub_df = pd.read_csv(f'data_files/water_velocities_{i}.csv')
    
    water_idx = sub_df['system:index'].str.split('_', expand=True)
    water_idx = water_idx.rename(columns={0:'cluster', 1:'index'})
    water_idx['cluster'] = water_idx['cluster'].astype(int) + i
    
    sub_df = sub_df.join(water_idx).drop(columns=['system:index'])
    
    df_list.append(sub_df)
    
water_df = pd.concat(df_list, axis=0)
water_df

,time,velocity_u_0,velocity_v_0,.geo,cluster,index
0,1.039478e+12,-282.0,-13.0,"{""type"":""Point"",""coordinates"":[-116.1746241187...",0,0
1,1.039565e+12,-233.0,25.0,"{""type"":""Point"",""coordinates"":[-116.1746241187...",0,1
2,1.039651e+12,-343.0,42.0,"{""type"":""Point"",""coordinates"":[-116.1746241187...",0,2
3,1.039478e+12,-282.0,-13.0,"{""type"":""Point"",""coordinates"":[-116.1297083545...",0,3
4,1.039565e+12,-233.0,25.0,"{""type"":""Point"",""coordinates"":[-116.1297083545...",0,4
...,...,...,...,...,...,...
21103,1.249344e+12,59.0,-247.0,"{""type"":""Point"",""coordinates"":[-131.4010681845...",1999,31
21104,1.249430e+12,0.0,-286.0,"{""type"":""Point"",""coordinates"":[-131.4010681845...",1999,32
21105,1.249258e+12,104.0,-116.0,"{""type"":""Point"",""coordinates"":[-131.3561524203...",1999,33
21106,1.249344e+12,63.0,-271.0,"{""type"":""Point"",""coordinates"":[-131.3561524203...",1999,34


In [105]:
water_df['.geo'] = water_df['.geo'].apply(json.loads).apply(shape)
water_df = gpd.GeoDataFrame(water_df, geometry='.geo')
water_df

,time,velocity_u_0,velocity_v_0,.geo,cluster,index
0,1.039478e+12,-282.0,-13.0,POINT (-116.17462 16.77604),0,0
1,1.039565e+12,-233.0,25.0,POINT (-116.17462 16.77604),0,1
2,1.039651e+12,-343.0,42.0,POINT (-116.17462 16.77604),0,2
3,1.039478e+12,-282.0,-13.0,POINT (-116.12971 16.77604),0,3
4,1.039565e+12,-233.0,25.0,POINT (-116.12971 16.77604),0,4
...,...,...,...,...,...,...
21103,1.249344e+12,59.0,-247.0,POINT (-131.40107 32.22706),1999,31
21104,1.249430e+12,0.0,-286.0,POINT (-131.40107 32.22706),1999,32
21105,1.249258e+12,104.0,-116.0,POINT (-131.35615 32.22706),1999,33
21106,1.249344e+12,63.0,-271.0,POINT (-131.35615 32.22706),1999,34


Some clusters will be NaN values, remove those.

In [106]:
water_df = water_df.dropna().set_index(['cluster'])
remap = dict(zip(water_df.index.unique(), list(range(len(water_df.index.unique())))))
water_df = water_df.rename(index=remap).set_index(['index'], append=True)
water_df

time  velocity_u_0  velocity_v_0  \
cluster index                                             
0       0      1.039478e+12        -282.0         -13.0   
        1      1.039565e+12        -233.0          25.0   
        2      1.039651e+12        -343.0          42.0   
        3      1.039478e+12        -282.0         -13.0   
        4      1.039565e+12        -233.0          25.0   
...                     ...           ...           ...   
1946    31     1.249344e+12          59.0        -247.0   
        32     1.249430e+12           0.0        -286.0   
        33     1.249258e+12         104.0        -116.0   
        34     1.249344e+12          63.0        -271.0   
        35     1.249430e+12          23.0        -304.0   

                                      .geo  
cluster index                               
0       0      POINT (-116.17462 16.77604)  
        1      POINT (-116.17462 16.77604)  
        2      POINT (-116.17462 16.77604)  
        3      POINT (-116.12971 16.77604)  
        4      POINT (-116.12971 16.77604)  
...                                    ...  
1946    31     POINT (-131.40107 32.22706)  
        32     POINT (-131.40107 32.22706)  
        33     POINT (-131.35615 32.22706)  
        34     POINT (-131.35615 32.22706)  
        35     POINT (-131.35615 32.22706)  

[79994 rows x 4 columns]

In [109]:
images = []
largestx, largesty = -1, -1
for key, cluster_df in water_df.groupby(level=0):
    cluster_df = cluster_df.droplevel(0)

    cluster_df.time = pd.Categorical(cluster_df.time)
    cluster_df.time = cluster_df.time.cat.rename_categories(list(range(len(cluster_df.time.cat.categories))))

    cluster_df['x'] = pd.Categorical(cluster_df.geometry.x)
    cluster_df['y'] = pd.Categorical(cluster_df.geometry.y)
    cluster_df.x = cluster_df.x.cat.rename_categories(list(range(len(cluster_df.x.cat.categories))))
    cluster_df.y = cluster_df.y.cat.rename_categories(list(range(len(cluster_df.y.cat.categories))))

    time, maxx, maxy = cluster_df.time.cat.categories[-1] + 1, cluster_df.x.cat.categories[-1] + 1, cluster_df.y.cat.categories[-1] + 1

    cluster_df = cluster_df.pivot_table(values=['velocity_u_0', 'velocity_v_0'], index=['time', 'y', 'x'], aggfunc=np.sum)
    img = cluster_df.to_numpy().reshape([time, maxy, maxx, -1])
    largestx, largesty = max(maxx, largestx), max(maxy, largesty)
    
    # [labels, img]
    images.append([df.loc[key, 'Density'], img])
    
len(images), largestx, largesty

(1947, 11, 10)

In [110]:
with open('train_data.pickle', 'wb') as handle:
    pickle.dump(images, handle, protocol=pickle.HIGHEST_PROTOCOL)

Test padding an image.

In [136]:
img = images[0][1]
img.shape

(3, 5, 5, 2)

In [137]:
height, width = 11, 11
# calculate margins for paddings
top = int(np.floor((height - img.shape[1])/2.0))
bottom = int(np.ceil((height - img.shape[1])/2.0))
left = int(np.floor((width - img.shape[2])/2.0))
right = int(np.ceil((width - img.shape[2])/2.0))

# pad the image -> [don't pad time, pad height, pad width, don't pad channels]
img = np.pad(img, [(0, 0), (top, bottom), (left, right), (0, 0)], 'constant', constant_values=0)
img.shape

(3, 11, 11, 2)